<a href="https://colab.research.google.com/github/RYU-MCFLY/Algebra_lineal-para-ML/blob/main/Semana3_2_Aps_Financieras5_LSTM_Bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Aplicaciones-Financieras/blob/main/Semana3/2_LSTM_Bitcoin.ipynb)



# Descripción del problema

Datos originales: https://challengedata.ens.fr/participants/challenges/31/

El problema trata de buscar un algoritmo de clasificación que ayude a crear estrategias de inversión en criptomonedas, basado en el "sentimiento" extraído de noticias y redes sociales.

Por cada hora de trading se contabilizó la ocurrencia de algunos terminos, tales como 'adoption' y 'hack', en un selecto numero de cuentas influyentes de twitter y en algunos foros como 'Bitcointalk'.

Se han creado 10 temas diferentes, algunos positivos y otros negativos y se han contabilizado las palabras antes mencionadas, antes de una normalización.

Dado un tema y un tema, hemos visto los conteos de las últimas 48 horas y se estandarizaron esos conteos. El resultado se multiplicó por el conteo promedio por hora y se dividió por el conteo promedio por hora de todo el entrenamiento

Para un tiempo T en el periodo de tiempo i, con lag k ($k\in[\![0;47]\!]$) el valor F ode la característica será:

$$
F_{i,k}=\frac{T_{i,k}-\overline{T_{i}}}{\sqrt{\frac{1}{47}\sum\limits_{j=0}^{47}{(T_{i,j}-\overline{T_{i}})^{2}}}}*\frac{\overline{T_i}}{\overline{T}} 
$$


Se agregaron 5 características correspondientes a los precios finales en periodos de 1 hr, 6 hrs, 12 hrs, 24 hrs y 48 hrs
El objetivo es predecir si el precio del Bitcoin tendrá un retorno (en la próxima hora) que sea de mas del 0.2%, entre -0.2% y 0.2% o menos al -0.2%.

La métrica utilizada para la perdida es la perdida logistica, definita como el negativo de la log-verosimilitud de las etiquetas verdaderas comparadas con las probabilidades predichas por el clasificador.

Las verdaderas etiquetas están codificadas como una matríz de 3 columnas, donde hay unos o ceros dependiendo si el elemento pertenece a la categoría de una columna u otra.
 
Dada una matriz P de probabilidades $p_{i,k}=Pr(t_{i,k}=1)'$ , la función de perdida se define como

$$
L_{log}(Y,P)=-log{Pr(Y|P)}=-\frac{1}{N} \sum_{i=1}^{N} \sum_{k=1}^3{y_{i,k}log(p_{i,k})}
$$

Entre más bajo el score de ésta medida, mejor.



# Dependencias

In [ ]:
# !pip install -U plotly

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import layers

import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Funciones

In [ ]:
def evaluate_model(estimator, train, val, test):
    print('train cross_entropy = ', estimator.evaluate(train[0], train[1], verbose = False))
    print('  val cross_entropy = ', estimator.evaluate(val[0], val[1], verbose = False))
    print(' test cross_entropy = ', estimator.evaluate(test[0], test[1], verbose = False))

In [ ]:
# TODO: Modificar para seleccionar características cambiando n_features, no sólo las primeras n_features
# TODO: Revisar los resultados generados cuando se seleccionan distintos parámetros
# NOTE: Asuma que el dataframe tiene 5 X, y 48 columnas para cada una de las 10 I ordenadas de reciente a antigua
def transform_dataframe(df, len_prices = 5, n_features = 10, len_features = 48):
    if type(len_prices) != int or type(n_features) != int or type(len_features) != int:
        raise ValueError(f'Los parámetros len_prices, n_features y len_features deben ser de tipo int. Recibibo {type(len_prices)},{type(n_features)} y {type(len_features)}')

    assert 0 < len_prices <= 5, 'len_prices debe estar entre 1 y 5'
    assert 0 < n_features <= 10, 'n_features debe estar entre 1 y 10'
    assert 0 < len_features <= 48, 'len_features debe estar entre 1 y 48'

    df.reset_index(inplace = True, drop = True)
    
    # Los nombres de las columnas están al reves para tener primer la observación más antigua
    prices_cols = ['X5', 'X4', 'X3', 'X2', 'X1']

    prices = np.zeros((len(df), len_prices, 1))
    features = np.zeros((len(df), len_features, n_features))

    for i in range(len(df)):
        # Se transforman la forma de los precios
        prices[i] = df.loc[i, prices_cols[-len_prices:]].values.reshape((len_prices, 1))
        # Para cada característica
        for j in range(n_features):
            # Se obtiene los 48 rezagos y se voltea el arreglo para tener el más antiguo primero
            # Aquí se aplica el supuesto de que el dataframe tiene 5 columnas de 5
            features[i, :, j] = np.flip(df.iloc[i, 5+48*j:5+len_features + 48*j].values)
    return prices, features

> The Input data contains 10 time series of 48 trading hours representing complementary features based on sentiment analysis from news extracted from twitter or forums like Bitcointalk on Bitcoin, and 5 time series based on the variation of Bitcoin price during the past 1, 6, 12, 24 and 48 hours normalised by volatility during the period. Input data, for training and testing, will be given by a .csv file, whose first line contains the header. Then each line corresponds to a sample, each column to a feature. The features are the following:

>- ***ID***: Id of the sample which is linked to the ID of the output file;
- ***I_1_lag(k)*** to ***I_10_lag(k)***: Values of Indicators *I_1* to *I_10* for each k lag ($k\in[\![0;47]\!]$) representing the normalized value of Indicators *I_1* to *I_10* each hour of the past 48 trading hours;
- ***X_1*** to ***X_5***: Values of 5 normalised indicators representing price variation of Bitcoin on the last 1, 6, 12, 24 and 48 hours.

> There will be 14 000 samples for the train set and 5 000 for the test set. For a given sample, the time series (for the 10 sentiment indicators) are given over the same 48 trading hours.

>The training outputs are given in a .csv file. Each line corresponds to a sample:

>- ***ID***: Id of the sample;
- ***Target_-1***: classification of the return of Bitcoin in the next hour. -1 signifies a down move of less than -0.2%;
- ***Target_0***: classification of the return of Bitcoin in the next hour. 0 signifies a move between -0.2% and 0.2%;
- ***Target_1***: classification of the return of Bitcoin in the next hour. 1 signifies a up move of more than 0.2%.



In [ ]:
X_raw = pd.read_csv('/content/drive/MyDrive/Cruso-ApsFinancieras/semana7/input_training_IrTAw7w.csv').set_index('ID')
X_raw

In [ ]:
X_raw.loc[:,'I1_lag0':'I1_lag47']

In [ ]:
y_raw = pd.read_csv('/content/drive/MyDrive/Cruso-ApsFinancieras/semana7/output_training_F2dZW38.csv').set_index('ID')
y_raw

## División: Entrenamiento, Validación y Prueba

In [ ]:
# Division para entrenamiento de red LSTM
X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, train_size = .8, random_state = 10, shuffle = False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = .75, random_state = 10, shuffle = False)

print('     Train shape', X_train.shape)
print('Validation shape', X_val.shape)
print('      Test shape', X_test.shape)

## Transformaciones

In [ ]:
# Longitud de la serie de tiempo de variaciones de los precios del Bitcoin X_i
len_prices = 5

# Cantidad de temas vistos
n_features = 10
# Cantidad de tiempos que vemos hacia atras en las variables de sentimiento
len_features = 48

train_prices, train_features = transform_dataframe(X_train, len_prices, n_features, len_features)
val_prices, val_features     = transform_dataframe(X_val,   len_prices, n_features, len_features)
test_prices, test_features   = transform_dataframe(X_test,  len_prices, n_features, len_features)

In [ ]:
train_prices[0]

In [ ]:
train_prices.shape

In [ ]:
train_features[0][:, 0]

In [ ]:
X_raw.loc[:,'I1_lag0':'I1_lag47']

In [ ]:
train_features.shape

In [ ]:
print('        Labels: (samples, sequence length, features)')
print('  Train prices:', train_prices.shape)
print('Train features:', train_features.shape)

# Algoritmos-Modelos

## Regresión logística

El punto de referencia de los propietarios de los datos es una regresión logística tomando como características X1, $\dots$, X5.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
np.set_printoptions(suppress=True)
train_prices

In [ ]:
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_raw, y_raw, train_size = .8, random_state = 10, shuffle = False)

In [ ]:
X_train_0 = X_train_0[['X1', 'X2', 'X3', 'X4', 'X5']]
X_train_0

In [ ]:
X_test_0 = X_test_0[['X1', 'X2', 'X3', 'X4', 'X5']]
X_test_0

In [ ]:
y_train_0

In [ ]:
y_train_0 = y_train_0.idxmax(axis=1)
y_train_0

In [ ]:
y_test_0 = y_test_0.idxmax(axis=1)
y_test_0

In [ ]:
model_0 = LogisticRegression(multi_class='multinomial', random_state=1)

In [ ]:
model_0

In [ ]:
model_0.fit(X_train_0, y_train_0 )

In [ ]:
y_pred_0 = model_0.predict(X_test_0)

In [ ]:
(y_pred_0 == y_test_0).sum() / len(y_pred_0)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_0, y_pred_0))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test_0, y_pred_0)

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
# Otra salida útil del modelo
model_0.predict_proba(X_test_0)

In [ ]:
y_test_0

In [ ]:
log_loss(y_test_0, model_0.predict_proba(X_test_0))

## Redes Neuronales

#### Callbacks

In [ ]:
# Los guarda en GoogleDrive, si no se quiere hacer la conexión, entonces cambiar la ruta o quitar el callback
# checkpoint_cb = keras.callbacks.ModelCheckpoint(
#     filepath = 'checkpoints', 
#     save_best_only = True
# )

earlystop_cb = keras.callbacks.EarlyStopping(
    patience = 25, 
    restore_best_weights = True
)

### Primera Aquitectura (Solo utilizando Caracteristicas de Bitcoin)

#### Aquitectura

In [ ]:
# Definicion de las capas
price_input = layers.Input(shape = (len_prices, 1), name = 'price_input')
price_normalization = layers.Normalization(name = 'price_normalization', axis = 1)
price_lstm = layers.LSTM(32, name = 'price', 
                        # kernel_regularizer=keras.regularizers.l1_l2(l1 = 1e-1, l2 = 1e-1),
                         return_sequences=True, 
                         )

price_lstm2 = layers.LSTM(16, name='price2')

# Como se conectan las capas definidas arriba
price_normalized = price_normalization(price_input)
price_output = price_lstm(price_normalized)
price_output2 = price_lstm2(price_output)

# Como se conecta lo posterior
prediction_layer = keras.models.Sequential(
    [
     layers.Dense(32, activation = 'relu'), 
     layers.Dense(32, activation = 'relu'), 
     layers.Dense(32, activation = 'relu'), 
     layers.Dense(3, activation = 'softmax')
    ], 
    name = 'prediction'
)
prediction_output = prediction_layer(price_output2) 


#### Modelo

In [ ]:
# Instanciación del objeto modelo
model = keras.Model(price_input, prediction_output)

model.compile(
    loss = 'categorical_crossentropy'
)

model.summary()

##### Visualización

In [ ]:
# Visualización del modelo
keras.utils.plot_model( 
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TD",
    dpi=100,
)

#### Entrenamiento

In [ ]:
history = model.fit(
    train_prices, y_train,
    batch_size = 128,
    epochs = 1000,  # Valor real de 1000 
    callbacks = [earlystop_cb], 
    validation_data = [val_prices, y_val]
).history

##### Visualización

In [ ]:
x = list(range(len(history['loss'])))

fig = go.Figure()

fig.add_trace(go.Scatter(x = x, y = history['loss'])) \
   .add_trace(go.Scatter(x = x, y = history['val_loss']))

fig.show()

#### Métricas

In [ ]:
evaluate_model(model, [train_prices, y_train], [val_prices, y_val], [test_prices, y_test])

In [ ]:
y_pred_prob = model.predict(test_prices)

In [ ]:
y_pred_prob

In [ ]:
y_pred = y_pred_prob.argmax(axis=1)
y_pred

In [ ]:
y_test_f = y_test.idxmax(axis=1).map({'Target -1': 0, 'Target 0': 1, 'Target 1': 2})

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test_f, y_pred)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test_0, y_pred_0)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_f, y_pred))

### Segunda Arquitectura

#### Arquitectura

In [ ]:
fusion_dim = 8

price_input = layers.Input(shape = (len_prices,), name = 'price_input')

price_layer = keras.models.Sequential(
    [
     layers.Normalization(name = 'price_normalization'), 
     layers.Dense(8, activation = 'tanh'), 
     layers.Dense(8, activation = 'relu'),
     layers.Dense(8, activation = 'tanh'), 
     layers.Dense(fusion_dim, activation = 'relu'),
    ], 
    name = 'price'
)

price_output = price_layer(price_input)

sentiment_input = layers.Input(shape = (len_features, n_features), name = 'sentiment_input')
sentiment_normalization = layers.Normalization(name = 'sentiment_normalization', axis = 1)
sentiment_lstm = layers.LSTM(fusion_dim, name = 'sentiment')

sentiment_normalized = sentiment_normalization(sentiment_input)
sentiment_output = sentiment_lstm(sentiment_normalized)

fusion = layers.Multiply(name = 'hadamard_product')([sentiment_output, price_output])

prediction_layer = keras.models.Sequential(
    [
     layers.Dense(8, activation = 'tanh'), 
     layers.Dense(8, activation = 'relu'), 
     layers.Dense(8, activation = 'sigmoid'), 
     layers.Dense(3, activation = 'softmax')
    ], 
    name = 'prediction'
)
prediction_output = prediction_layer(fusion)

#### Modelo

In [ ]:
model_2 = keras.Model([price_input, sentiment_input], prediction_output)

model_2.compile(
    loss = 'categorical_crossentropy'
)

model_2.summary()

##### Visualización

In [ ]:
keras.utils.plot_model( 
    model_2,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TD",
    dpi=90,
)

#### Entrenamiento

In [ ]:
history = model_2.fit(
    [train_prices, train_features], y_train, 
    epochs = 30, # Original, 1000 épocas
    callbacks = [earlystop_cb], 
    validation_data = [[val_prices, val_features], y_val]
).history

##### Visualización

In [ ]:
x = list(range(len(history['loss'])))

fig = go.Figure()

fig.add_trace(go.Scatter(x = x, y = history['loss'])) \
   .add_trace(go.Scatter(x = x, y = history['val_loss'])) \
   .update_layout(hovermode = 'x')

fig.show()

#### Métricas

In [ ]:
evaluate_model(model_2, [[train_prices, train_features], y_train], [[val_prices, val_features], y_val], [[test_prices, test_features], y_test])

## Guardando el mejor modelo

In [ ]:
model_2.save('best_model.h5')

# Cargar el modelo guardado

In [ ]:
best_model = keras.models.load_model('best_model.h5')
best_model.summary()

In [ ]:
evaluate_model(best_model, [[train_prices, train_features], y_train], [[val_prices, val_features], y_val], [[test_prices, test_features], y_test])

# Resumen

Respecto a la segunda arquitectura:

1. Fue la que tuvo mejores resultados con entropía cruzada de $0.9843$, $1.0256$ y $1.0329$ para los conjuntos de entrenamiento, validación y prueba, respectivamente.
2. Tardó menos épocas en entrenar



# Ejercicios

1. Crear un nuevo modelo ya sea haciendo modificaciones a alguna de las arquitecturas presentes, o comenzando desde cero. Escriba sus observaciones comparando el resultado con el mejor la Segunda Arquitectura. Como sugerencia, puede probar cambiando los datos de entrada, las capas de las redes secuenciales, la fusión de los datos, agregando regularización o dropout, callbacks, etc. 
2. ¿Cómo haría la predicción para nuevos valores?

3. Cree las matrices de confusión para la última red neuronal, así como el classification report.
